# Predictive Analytics in Horse Racing using Gradient Boosting and AdaBoost

### *Group Members*

- Harrye Fredericksen (23020603)
- Kobe Spring (24004428)
- Lagi Rabo (04225368)
- Jason Wing (16339768)

## Introduction


Horse racing 

### ***Imports***

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, f1_score, make_scorer
import matplotlib.pyplot as plt


### ***Dataset***



In [2]:
#Harrye please show the code that was used to obtain this data and place in any cleaning code that was performed - make sure to have it documented down here as EXTENSIVELY as possible please.

In [3]:
cleaned_df = pd.read_csv("cleaned_data_v1.csv")
cleaned_df.head(10)

,course_id,race_id,dist_m,going,ran,pos,draw,ovr_btn,btn,horse_id,...,has_blinkers,has_tongue_tie,has_visor,has_cheekpieces,has_eye_hood,has_eyecover,has_eyeshield,has_hood,no_headgear,first_time_equipment
0,297,308856,3200,5.0,22,1,13.0,0.00,0.00,548204,...,0,0,0,0,0,0,0,0,0,0
1,297,308856,3200,5.0,22,2,15.0,0.75,0.75,518856,...,0,0,0,0,0,0,0,0,0,0
2,297,308856,3200,5.0,22,3,11.0,6.75,6.00,447498,...,0,0,0,0,0,0,0,0,0,0
3,297,308856,3200,5.0,22,4,19.0,7.50,0.75,458259,...,1,0,0,0,0,0,0,0,0,0
4,297,308856,3200,5.0,22,5,10.0,7.75,0.30,554624,...,0,0,0,0,0,0,0,0,0,0
5,297,308856,3200,5.0,22,6,23.0,7.75,0.05,554625,...,0,0,0,0,0,0,0,0,0,0
6,297,308856,3200,5.0,22,7,7.0,8.50,0.75,519815,...,1,0,0,0,0,0,0,0,0,0
7,297,308856,3200,5.0,22,8,21.0,8.75,0.20,504666,...,1,0,0,0,0,0,0,0,0,0
8,297,308856,3200,5.0,22,9,17.0,10.25,1.50,529388,...,0,0,0,0,0,0,0,0,0,0
9,297,308856,3200,5.0,22,10,3.0,12.00,1.75,514914,...,0,0,0,0,0,0,0,0,0,0


### Harrye

In [4]:
#Under your name enter in the ML code that you have created - ADDITIONALLY PLEASE write a relatively extensive paragraph AT minimum 5-6 sentences long that outlines what your ML question is looking at and why etc. I recommend everyone mnake their own initial copy of the df please.

### Kobe


In [5]:
#notes: discovered that there we a few very prolific sires so therefore had to log transform. one sire had 1000< foals - whilst alot have less <10

In [29]:
kns_df = cleaned_df.copy()
kns_df['top3'] = (kns_df['pos'] <= 3).astype(int)

kns_train_df, kns_test_df = train_test_split(
    kns_df, test_size=0.2, stratify=kns_df['top3'], random_state=42
)

kns_min_runners = 3

kns_sire_stats = (
    kns_train_df.groupby('sire_id')['top3']
    .agg(['mean', 'count'])
    .rename(columns={'mean': 'sire_top3_rate', 'count': 'sire_runners'})
)
kns_sire_stats = kns_sire_stats[kns_sire_stats['sire_runners'] >= kns_min_runners]

kns_dam_stats = (
    kns_train_df.groupby('dam_id')['top3']
    .agg(['mean', 'count'])
    .rename(columns={'mean': 'dam_top3_rate', 'count': 'dam_runners'})
)
kns_dam_stats = kns_dam_stats[kns_dam_stats['dam_runners'] >= kns_min_runners]

kns_train_df = (
    kns_train_df
    .merge(kns_sire_stats[['sire_top3_rate','sire_runners']], on='sire_id', how='left')
    .merge(kns_dam_stats[['dam_top3_rate','dam_runners']], on='dam_id', how='left')
)

kns_test_df = (
    kns_test_df
    .merge(kns_sire_stats[['sire_top3_rate','sire_runners']], on='sire_id', how='left')
    .merge(kns_dam_stats[['dam_top3_rate','dam_runners']], on='dam_id', how='left')
)

for kns_col in ['sire_top3_rate','dam_top3_rate','sire_runners','dam_runners']:
    kns_train_df[kns_col] = kns_train_df[kns_col].fillna(0)
    kns_test_df[kns_col]  = kns_test_df[kns_col].fillna(0)

In [30]:
for kns_df_tmp in [kns_train_df, kns_test_df]:
    kns_df_tmp['sire_runners_log'] = np.log1p(kns_df_tmp['sire_runners'])

kns_base_feats = ['dist_m','going','ran','age','lbs','has_blinkers','has_tongue_tie']

kns_lineage_feats = kns_base_feats + ['sire_top3_rate','dam_top3_rate','sire_runners_log','dam_runners']

kns_categorical_feats = ['has_blinkers', 'has_tongue_tie']

kns_base_feats    = [f for f in kns_base_feats if f in kns_train_df.columns]
kns_lineage_feats = [f for f in kns_lineage_feats if f in kns_train_df.columns]

kns_X_train_b, kns_y_train_b = kns_train_df[kns_base_feats],    kns_train_df['top3']
kns_X_test_b,  kns_y_test_b  = kns_test_df[kns_base_feats],     kns_test_df['top3']

kns_X_train_l, kns_y_train_l = kns_train_df[kns_lineage_feats], kns_train_df['top3']
kns_X_test_l,  kns_y_test_l  = kns_test_df[kns_lineage_feats],  kns_test_df['top3']


In [33]:
def run_lightgbm(X_train, y_train, X_test, y_test, feature_label):

    print(f"\n*** Running LightGBM ({feature_label.upper()} FEATURES) ***\n")

    scale = (y_train == 0).sum() / (y_train == 1).sum()
    print(f"Scale_pos_weight = {scale:.2f}\n")

    n_estimators  = [100, 200, 300, 400, 500]
    learning_rate = [0.005, 0.01, 0.05, 0.1]
    max_depth     = [-1, 3, 5]

    best_auc  = 0
    best_combo = None

    for n in n_estimators:
        for lr in learning_rate:
            for md in max_depth:

                model = LGBMClassifier(
                    n_estimators=n,
                    learning_rate=lr,
                    max_depth=md,
                    scale_pos_weight=scale,
                    random_state=42,
                    verbose=-1
                )

                model.fit(
                    X_train, y_train,
                    eval_set=[(X_test, y_test)],
                    eval_metric="auc",
                    callbacks=[lgb.early_stopping(50)],
                )

                preds_proba = model.predict_proba(X_test)[:, 1]
                preds_label = (preds_proba >= 0.25).astype(int)

                acc = accuracy_score(y_test, preds_label)
                f1  = f1_score(y_test, preds_label)
                auc = roc_auc_score(y_test, preds_proba)

              #  print(f"Params (n_estimators={n}, learning_rate={lr}, max_depth={md}): "
                 #     f"AUC={auc:.4f}, ACC={acc:.4f}, F1={f1:.4f}")

                if auc > best_auc:
                    best_auc  = auc
                    best_combo = (n, lr, md)

    print(f"\nBest LightGBM ({feature_label}) model: AUC={best_auc:.4f} with params {best_combo}\n")
    return best_auc, best_combo

In [34]:
base_auc, base_params = run_lightgbm(
    kns_X_train_b, kns_y_train_b, kns_X_test_b, kns_y_test_b, feature_label="base"
)

lineage_auc, lineage_params = run_lightgbm(
    kns_X_train_l, kns_y_train_l, kns_X_test_l, kns_y_test_l, feature_label="lineage"
)

print(f"\nAUC (base): {base_auc:.4f}")
print(f"AUC (lineage): {lineage_auc:.4f}")
print(f"Improvement: {lineage_auc - base_auc:.4f}")


*** Running LightGBM (BASE FEATURES) ***

Scale_pos_weight = 2.84

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[29]	valid_0's auc: 0.615437	valid_0's binary_logloss: 0.570396
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[25]	valid_0's auc: 0.610697	valid_0's binary_logloss: 0.571095
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[29]	valid_0's auc: 0.615085	valid_0's binary_logloss: 0.570545
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[14]	valid_0's auc: 0.615441	valid_0's binary_logloss: 0.570385
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[12]	valid_0's auc: 0.611098	valid_0's binary_logloss: 0.571093
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[14]	valid_0's auc: 0.614973	valid_0

### Lagi

In [ ]:
#Under your name enter in the ML code that you have created - ADDITIONALLY PLEASE write a relatively extensive paragraph AT minimum 5-6 sentences long that outlines what your ML question is looking at and why etc.  I recommend everyone mnake their own initial copy of the df please.

### Jason

**Research Question:** "Do frequent trainer/jockey pairings outperform infrequent pairings?"

In the context of horse racing, a trainer and jockey need to work closely together to bring the best performance out of the horse. It is reasonable to therefore assume that ongoing relationships between jockey/trainer would equate to better performance. 

The purpose of this research question is to establish whether there is a relationship between frequent pairings and performance, and to what extent this relationship may be useful in predicting race outcomes. 

As outlined in the exploration below, I have defined a 'frequent pairing' as a jockey/trainer pairing that has worked together 10 or more times which captures approximately 6.5% of pairings.  